In [1]:
import importlib
import sys
sys.path.insert(0, '../src')

In [2]:
import torch
import torch.nn as nn
import os
import json
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import pyarrow.parquet as pq
import numpy as np
import torch.nn.functional as F
from tqdm import tqdm
import pytorch_lightning as pl
import sys
# from sklearn import *
from torchmetrics.classification import accuracy

In [3]:
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint, DeviceStatsMonitor, TQDMProgressBar,EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.tuner import Tuner
from config import *
from data import data_utils
from data.dataset import ASL_DATASET
from dl_utils import get_dataloader

2023-06-05 00:08:51.812114: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
DL_FRAMEWORK

'pytorch'

In [5]:
module_name = f"models.{DL_FRAMEWORK}.lightning_models"
class_name = "LightningTransformerPredictor"

In [6]:
module = importlib.import_module(module_name)
MyTransformerModel = getattr(module, class_name)
MyTransformerModel

models.pytorch.lightning_models.LightningTransformerPredictor

In [7]:
DEVICE

'mps'

In [8]:
class MyProgressBar(TQDMProgressBar):
    def init_validation_tqdm(self):
        bar = super().init_validation_tqdm()
        if not sys.stdout.isatty():
            bar.disable = True
        return bar

    def init_predict_tqdm(self):
        bar = super().init_predict_tqdm()
        if not sys.stdout.isatty():
            bar.disable = True
        return bar

    def init_test_tqdm(self):
        bar = super().init_test_tqdm()
        if not sys.stdout.isatty():
            bar.disable = True
        return bar

In [9]:
# Define parameters
params = {
    "d_model": 192,
    "n_head": 8,
    "dim_feedforward": 512,
    "dropout": 0.1,
    "layer_norm_eps": 1e-5,
    "norm_first": True,
    "batch_first": True,
    "num_layers": 2,
    "num_classes": 250,  
    "learning_rate": 0.001, 
}

In [10]:
asl_dataset = ASL_DATASET(augment=True, augmentation_threshold=0.3)

In [11]:
train_ds, val_ds, test_ds = data_utils.create_data_loaders(asl_dataset,
                                                               batch_size=BATCH_SIZE,
                                                               dl_framework=DL_FRAMEWORK,
                                                               num_workers=4)

In [12]:
print(f'Got the lengths for Train-Dataset: {len(train_ds)}, {len(val_ds)}, {len(test_ds)}')

Got the lengths for Train-Dataset: 484, 29, 57


In [13]:
batch = next(iter(train_ds))[0]
batch.shape

torch.Size([128, 32, 96, 2])

In [16]:
model = MyTransformerModel(**params).to(DEVICE)
model(batch.to(DEVICE))
model = model.float().to(DEVICE)
print(model)

LightningTransformerPredictor(
  (criterion): CrossEntropyLoss()
  (accuracy): MulticlassAccuracy()
  (model): LightningTransformerSequenceClassifier(
    (transformer): TransformerEncoder(
      (layers): ModuleList(
        (0-1): 2 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=192, out_features=192, bias=True)
          )
          (linear1): Linear(in_features=192, out_features=512, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=512, out_features=192, bias=True)
          (norm1): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
      )
      (norm): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
    )
    (output_layer): Linear(in_features=1

In [17]:
checkpoint_callback = ModelCheckpoint(
        filename=class_name + "-{epoch:02d}-{val_accuracy:.2f}",
        save_top_k=1,
        monitor="val_accuracy",
        verbose=True,
        mode="max"
    )

In [18]:
tb_logger = TensorBoardLogger(
        save_dir=os.path.join(ROOT_PATH, "lightning_logs"),
        name=class_name,
        # version=mod_name
    )

In [19]:
early_stop_callback = EarlyStopping(
        monitor='val_accuracy',
        min_delta=0.005,
        patience=6,
        verbose=True,
        mode='max'
    )

In [20]:
lr_monitor = LearningRateMonitor(logging_interval='step')

In [21]:
trainer = pl.Trainer(
        enable_progress_bar=True,
        accelerator="gpu",
        logger=tb_logger,
        callbacks=[
            DeviceStatsMonitor(),
            early_stop_callback,
           checkpoint_callback,
           MyProgressBar(),
            lr_monitor
        ],
        max_epochs=100,
       # limit_train_batches=10,
        # limit_val_batches=0,
        num_sanity_val_steps=0,
        profiler=None,  # select from None
    )

MisconfigurationException: No supported gpu backend found!

In [ ]:
trainer.fit(
        model=model,
        train_dataloaders=train_ds,
        val_dataloaders=val_ds,

    )



  | Name      | Type                                   | Params
---------------------------------------------------------------------
0 | criterion | CrossEntropyLoss                       | 0     
1 | accuracy  | MulticlassAccuracy                     | 0     
2 | model     | LightningTransformerSequenceClassifier | 741 K 
---------------------------------------------------------------------
741 K     Trainable params
0         Non-trainable params
741 K     Total params
2.965     Total estimated model params size (MB)


In [34]:
trainer.test(ckpt_path="best",
                 dataloaders=test_ds
                 )

In [1]:
import tensorflow as tf
from tensorflow_addons.layers import MultiHeadAttention, PositionalEncoding


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


ImportError: cannot import name 'PositionalEncoding' from 'tensorflow_addons.layers' (/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tensorflow_addons/layers/__init__.py)